In [1]:
qi("AmpelContext", "AmpelDB", "AlertProcessor", "TarAlertLoader")

In [2]:
ctx = AmpelContext.load("ampel_conf.json", tier=2)
dict.__setitem__(ctx.config._config['db'], 'prefix', 'AmpelTest')
AmpelDB.delete_ampel_databases(ctx.config, "AmpelTest")

In [3]:
ap = AlertProcessor(
    context = ctx,
    publish_stats = ["mongo"],
    supplier = {
        "unit": "FilteringAlertSupplier",
        "config": {
            "supplier": {"unit": "ZiAlertSupplier"},
            "match_ids": [770239962715015024]
        }
    },
    log_profile = "default",
    process_name = "ap_filtering_supplier_test",
    directives = [
        {
            "channel": "VAL_TEST1", 
            "filter": {"unit": "RandFilter", "config": {"passing_rate": 1.}},
            "stock_update": "ZiStockIngester",
            't0_add': {
                "ingester": "ZiAlertContentIngester",
                "t1_combine": [
                    {
                        "ingester": "PhotoCompoundIngester",
                        "config": {"combiner": "ZiT1Combiner"},
                        "t2_compute": {
                            "ingester": "PhotoT2Ingester",
                            "config": {"tags": ["ZTF"]},
                            "units": [{'unit': 'T2SNCosmo', 'config': 55880712071630791}]
                        }
                    }
                ]
            }
        }
    ]
)


2020-06-18 16:58:32 AmpelDB:159 INFO
 Creating AmpelTest_data -> stock
   Creating index: {'index': [('_id', 1), ('channel', 1)], 'args': {'unique': True}}

2020-06-18 16:58:32 FilterBlock:77 INFO
 Loading filter: RandFilter

2020-06-18 16:58:32 AlertProcessor:175 INFO
 AlertProcessor setup completed


In [4]:
ap.set_loader(
    TarAlertLoader('/Users/hu/Documents/ZTF/ztf_public_20190210.tar.gz')
)
ap.run()


2020-06-18 16:58:32 AmpelDB:159 INFO
 Creating AmpelTest_ext -> counter

2020-06-18 16:58:32 AmpelDB:159 INFO
 Creating AmpelTest_var -> logs
   Creating index: {'index': [('run', 1)]}
   Creating index: {'index': [('stock', 1)], 'args': {'sparse': True}}
   Creating index: {'index': [('channel', 1)], 'args': {'sparse': True}}

2020-06-18 16:58:33 AmpelDB:159 INFO
 Creating AmpelTest_var -> events

2020-06-18 16:58:33 AmpelDB:159 INFO
 Creating AmpelTest_data -> t2
   Creating index: {'index': [('stock', 1), ('channel', 1)]}
   Creating index: {'index': [('status', 1)]}

2020-06-18 16:58:33 AmpelDB:159 INFO
 Creating AmpelTest_data -> t0
   Creating index: {'index': [('stock', 1)]}

2020-06-18 16:58:33 AmpelDB:159 INFO
 Creating AmpelTest_data -> t1
   Creating index: {'index': [('stock', 1)]}

2020-06-18 16:58:34 CORE AlertProcessor:347 SHOUT [r=1]
 Processing alerts

2020-06-18 16:58:34 CORE FilterBlock:150 INFO [stock=3824288, channel=VAL_TEST1, alert=770239962715015024]

2020-06-1

1